In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!pip install music21

    100% |████████████████████████████████| 18.0MB 2.1MB/s 
  Running setup.py bdist_wheel for music21 ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /root/.cache/pip/wheels/53/8b/a6/be1921c60a68f0bea31c6b6a0a7b125badd61294d6a694407f
Successfully built music21


In [1]:
import os

batch_size = 1
max_length = 300
lstm_dim = 512
latent_dim = 2
epochs = 1000

work_dir = '/Users/masatoshi/repos/keras_composer/'
out_dir = os.path.join(work_dir, 'vae_new_output_101')

In [2]:
import os
import glob
from music21 import converter, instrument, note, chord, stream

def parse_midi_files(dir):
    notes = []
    songs = []
    file_list = []
    
    files = glob.glob(os.path.join(dir, '*.mid'))

    for file in files:
        song = []
        
        file_list.append(os.path.basename(file))
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                song.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                song.append('.'.join(str(n) for n in element.normalOrder))
        songs.append(song)
        notes += song

    return notes, songs, file_list

In [3]:
from keras.preprocessing.sequence import pad_sequences

notes, songs, file_list = parse_midi_files(os.path.join(work_dir,'midi_songs'))

pitchnames = sorted(set(notes))
n_vocab = len(pitchnames)

note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
int_to_note = dict([[number, note] for note, number in note_to_int.items()])

encoded_songs = [[note_to_int[note] for note in song] for song in songs]

padded_songs = pad_sequences(encoded_songs, maxlen=max_length)

Using TensorFlow backend.
/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Parsing /Users/masatoshi/repos/keras_composer/midi_songs/bwv782.mid
Parsing /Users/masatoshi/repos/keras_composer/midi_songs/bwv783.mid
Parsing /Users/masatoshi/repos/keras_composer/midi_songs/bwv781.mid
Parsing /Users/masatoshi/repos/keras_composer/midi_songs/bwv780.mid
Parsing /Users/masatoshi/repos/keras_composer/midi_songs/bwv784.mid
Parsing /Users/masatoshi/repos/keras_composer/midi_songs/bwv785.mid
Parsing /Users/masatoshi/repos/keras_composer/midi_songs/bwv778.mid
Parsing /Users/masatoshi/repos/keras_composer/midi_songs/bwv786.mid
Parsing /Users/masatoshi/repos/keras_composer/midi_songs/bwv779.mid
Parsing /Users/masatoshi/repos/keras_composer/midi_songs/bwv774.mid
Parsing /Users/masatoshi/repos/keras_composer/midi_songs/bwv775.mid
Parsing /Users/masatoshi/repos/keras_composer/midi_songs/bwv777.mid
Parsing /Users/masatoshi/repos/keras_composer/midi_songs/bwv776.mid
Parsing /Users/masatoshi/repos/keras_composer/midi_songs/bwv772.mid
Parsing /Users/masatoshi/repos/keras_composer/mi

In [4]:
def prepare_sequences(notes, n_vocab, sequence_length=100):
    """ Prepare the sequences used by the Neural Network """
    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    
    # convert notes to one-hot encoded
    one_hot_notes = []
    for note in notes:
        one_hot_note = np.zeros(n_vocab)
        one_hot_note[note_to_int[note]] = 1
        one_hot_notes.append(one_hot_note)

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(one_hot_notes) - sequence_length, 1):
        sequence_in = one_hot_notes[i:i + sequence_length]
        sequence_out = one_hot_notes[i + sequence_length]
        network_input.append(sequence_in)
        network_output.append(sequence_out)

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, n_vocab))

    network_output = np.array(network_output)

    return (network_input, network_output)

In [ ]:
network_input, _ = prepare_sequences(notes, n_vocab, sequence_length=max_length)

In [5]:
from keras.models import Model
from keras.layers import Input, LSTM, CuDNNLSTM
from keras.layers.core import Dense

x = Input(shape=(max_length, n_vocab))

h = CuDNNLSTM(lstm_dim, return_sequences=False, name='lstm_1')(x)

z_mean = Dense(latent_dim)(h) # 潜在変数の平均 μ
z_log_var = Dense(latent_dim)(h) #潜在変数の分散 σのlog

encoder = Model(inputs=x, outputs=[z_mean, z_log_var])

In [6]:
from keras.utils import plot_model

plot_model(encoder, to_file=os.path.join(work_dir, 'vae_encoder.png'), show_shapes=True)

In [7]:
from keras import backend as K
from keras.layers import Lambda

# 潜在変数のサンプリング
# 平均と分散を受けて、潜在変数をサンプリングします。
# z = μ + εΣ
# εは標準正規分布
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0.,
                              stddev=1.0)
    return z_mean + K.exp(z_log_var) * epsilon

In [8]:
# Lambdaを使って式をwrap
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

In [9]:
from keras.layers import Dense, RepeatVector, TimeDistributed

decoder_input = Input(shape=(latent_dim,))

repeated_context = RepeatVector(max_length)(decoder_input)
h_decoded = CuDNNLSTM(lstm_dim, return_sequences=True, name='dec_lstm_1')(repeated_context)
decoder_output = TimeDistributed(Dense(n_vocab, activation='softmax'), name='decoded_mean')(h_decoded)

decoder = Model(inputs=decoder_input, outputs=decoder_output)

x_decoded = decoder(z)

In [10]:
plot_model(decoder, to_file=os.path.join(work_dir, 'vae_decoder.png'), show_shapes=True)

In [11]:
from keras.layers import Layer
from keras import metrics

class CustomVariationalLayer(Layer): # Layer classの継承
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)

    def vae_loss(self, x, x_decoded):
        x = K.flatten(x)
        x_decoded = K.flatten(x_decoded)
        xent_loss = max_length * metrics.binary_crossentropy(x, x_decoded) # 復元誤差: Reconstruction Error
        kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1) # 正則化項: KL Divergence
        return K.mean(xent_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        x_decoded = inputs[1]
        loss = self.vae_loss(x, x_decoded)
        self.add_loss(loss, inputs=inputs) # Layer class のadd_lossを利用
        return x # 実質的には出力は利用しない

In [12]:
y = CustomVariationalLayer()([x, x_decoded])
vae = Model(x, y) # xをinputにyを出力, 出力は実質関係ない
vae.compile(optimizer='rmsprop', loss=None) # CustomVariationalLayerで追加したLossを利用するのでここでのlossはNoneとする

In [13]:
plot_model(vae, to_file=os.path.join(work_dir, 'vae_all.png'), show_shapes=True)

In [14]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger(os.path.join(out_dir, 'music_vae_log.csv'))

In [ ]:
vae.save_weights(os.path.join(work_dir, 'vae_weights.h5'))

In [16]:
vae.load_weights(os.path.join(work_dir, 'vae_weights.h5'))

InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNN' with these attrs.  Registered devices: [CPU], Registered kernels:
  <no registered kernels>

	 [[Node: lstm_1/CudnnRNN = CudnnRNN[T=DT_FLOAT, direction="unidirectional", dropout=0, input_mode="linear_input", is_training=true, rnn_mode="lstm", seed=87654321, seed2=0](lstm_1/transpose, lstm_1/ExpandDims_1, lstm_1/ExpandDims_2, lstm_1/concat_1)]]

Caused by op 'lstm_1/CudnnRNN', defined at:
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/masatoshi/pyenvs/music/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/masatoshi/pyenvs/music/lib/python3.6/site-packages/trait

In [0]:
def create_midi(prediction_output, file_path):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp=file_path)

In [0]:
z0 = 0.0
z1 = 0.0
z_sample = np.array([[z0, z1]])
decoder_output = decoder.predict(z_sample)
decoded_indices = np.argmax(decoder_output, axis=2)
decoded_song = [int_to_note[index] for index in decoded_indices[0]]
create_midi(decoded_song, 
                os.path.join(out_dir, 'vae_new_1_{}_{}.mid'.format(z0, z1)))